In [2]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import os

from nilearn.plotting import plot_stat_map
from nilearn import datasets

from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.image import index_img
from nilearn.image import mean_img
from nilearn import image
from nilearn.reporting import make_glm_report
from sklearn.model_selection import LeaveOneGroupOut
from nilearn.decoding import Decoder

### Define TR and subs, runs

Note: this script is only for 1 subject, for all 6 runs

In [14]:
TR = 1.0

sub = str('001')

runs = [str(i) for i in range(1, 2)]
print (sub)

001


#### Load behavioral data

In [19]:
events = []

for run in runs:
    temp = pd.read_csv(f'sub-{sub}/ses-001/beh/sub-{sub}_task-localizer_run-{run}_events.csv')

    temp = temp[['onset', 'trial_type', 'duration']]
    # append to events list
    events.append(temp)

events

[    onset trial_type  duration
 0      15      house        15
 1      45       face        15
 2      75      house        15
 3     105       face        15
 4     135      house        15
 5     165       face        15
 6     195      house        15
 7     225       face        15
 8     255      house        15
 9     285       face        15
 10    315      house        15
 11    345       face        15]

In [29]:
# instantiate the glm
glm = FirstLevelModel(
    t_r=TR,
    mask_img=f'sub-{sub}/ses-001/func/sub-{sub}_ses-FreemanTRSDOM_task-localizer_run-1_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
    high_pass=0.008,
    smoothing_fwhm=8
    )

In [33]:
# load fmri images
base = f'sub-{sub}/ses-001/func/sub-{sub}_ses-FreemanTRSDOM_task-localizer_'
fmri_imgs = [f'{base}run-{i}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' for i in range(1,2)]
fmri_imgs

['sub-001/ses-001/func/sub-001_ses-FreemanTRSDOM_task-localizer_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']

In [36]:
# make sure same number of images and event files
assert len(events) == len(fmri_imgs), f'events and fmri images not same length: {len(events)} events != {len(fmri_imgs)} images'

In [38]:
# fit the glm model 
glm.fit(fmri_imgs, events=events)

EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
# compute contrast, save as z_map
z_map = glm.compute_contrast('Trial-Fix')
z_map

/Users/gabe/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/first_level.py:706: UserWarning: One contrast given, assuming it for all 6 runs
  warn(f'One contrast given, assuming it for all {int(n_runs)} runs')


In [ ]:
# nib save
nib.save(z_map, 'nibabel_sub-001_zmap.nii.gz')

In [ ]:
# for visualization, make background image
mean_img_ = mean_img(fmri_imgs[0])

# make a nice report of the results
report = make_glm_report(
    glm,
    contrasts=['Trial-Fix']
)
report.save_as_html('report.html')
report

NameError: name 'mean_img' is not defined